# 03 - Déploiement et Interface Gradio

Ce notebook montre comment déployer le modèle via une interface **Gradio** :
- Chargement des préprocesseurs (TF-IDF, OneHot)
- Chargement des modèles entraînés
- Construction de la fonction de prédiction
- Mise en place de l'interface interactive


## Import des librairies

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
import gradio as gr
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio\routes.py:26: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio_client\documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")


## Préprocesseurs

In [3]:
DATA_PATH = os.path.join('data', 'fiverr_gigs', 'fiverr_cleaned.csv')
df = pd.read_csv(DATA_PATH, encoding='UTF-8', low_memory=False)
vectoriseur_titre = TfidfVectorizer(max_features=2000, stop_words='english')
vectoriseur_titre.fit(df['Description'].fillna(''))
encodeur_niveau = OneHotEncoder(sparse=False, handle_unknown='ignore')
encodeur_niveau.fit(df['Niveau'].fillna('Inconnu').values.reshape(-1, 1))

c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

## Modèles

In [5]:
MODELS_DIR = 'models'
modele_reg = joblib.load(os.path.join(MODELS_DIR, 'mor_rf.pkl'))
modele_clf = joblib.load(os.path.join(MODELS_DIR, 'level_clf.pkl'))

## Fonction de prédiction

In [6]:
def predict(titre_service, niveau_vendeur, nombre_avis):
    vec_titre = vectoriseur_titre.transform([titre_service])
    vec_niveau = encodeur_niveau.transform([[niveau_vendeur]])
    vec_avis = sparse.csr_matrix(np.array([[int(nombre_avis)]]))
    X = sparse.hstack([vec_titre, sparse.csr_matrix(vec_niveau), vec_avis], format='csr')
    prix_pred, note_pred = modele_reg.predict(X)[0]
    idx_niveau = modele_clf.predict(X)[0]
    niveau_pred = encodeur_niveau.categories_[0][idx_niveau]
    prix = round(float(prix_pred), 2)
    note = round(float(note_pred), 2)
    return prix, note, niveau_pred

## Interface Gradio

In [7]:
iface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Textbox(label="Titre du service", placeholder="Ex: Je vais développer votre site web..."),
        gr.Dropdown(label="Niveau du vendeur", choices=encodeur_niveau.categories_[0].tolist()),
        gr.Number(label="Nombre d'avis", value=0)
    ],
    outputs=[
        gr.Number(label="Prix prédit (EUR)"),
        gr.Number(label="Note moyenne prédite"),
        gr.Textbox(label="Niveau du vendeur prédit")
    ],
    title="Démo Prédiction Fiverr",
    description="Prédiction du prix, de la note et du niveau du vendeur via un modèle ML et une interface Gradio."
)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------


Traceback (most recent call last):
  File "c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio\routes.py", line 442, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio\blocks.py", line 1392, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\gradio\blocks.py", line 1097, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\wamp64\www\ia.follaco.fr\.venv\Lib\site-packages\anyio\_backends\_asyncio.py", line 2470, in run_sync_in_worker_thread
    return awai